In [1]:
import os
import json
import pandas as pd

# Đường dẫn đến thư mục version_1
root_dir = './data/version_1/kenh14/log_dir/recommend/new/'

# Tạo danh sách chứa thông tin từ các file config.json và test_sum_up.txt
data = []
all_files = []
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        # Kiểm tra nếu tên file là config.json hoặc test_sum_up.txt
        if file in ['config.json', 'test_sum_up.txt']:
            # Đường dẫn đến file
            file_path = os.path.join(subdir, file)
            # Đọc nội dung file
            with open(file_path, 'r') as f:
                content = f.read()
            # Nếu file là config.json thì parse nội dung json
            if file == 'config.json':
                content = json.loads(content)
            # Thêm thông tin vào danh sách
            data.append({
                'file_name': file,
                'file_path': file_path,
                'content': content
            })
            all_files.append(file_path)

# Tạo DataFrame từ danh sách
df = pd.DataFrame(data)

# Lưu DataFrame vào file csv
df.to_csv('output.csv', index=False)

In [2]:
all_dirs = ['/'.join(f.split('/')[:-1]) for f in all_files]

In [3]:
all_dirs = list(set(all_dirs))

In [4]:
valid_path = []
for d in all_dirs:
    try:
        for f in ['test_sum_up.txt', 'config.json']:
            file_path = os.path.join(d, f)
            with open(file_path, 'r') as f:
                content = f.read()
            # Nếu file là config.json thì parse nội dung json
            if file == 'config.json':
                content = json.loads(content)
            valid_path.append(file_path)
    except:
        print(file_path)

./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_28_26/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_06 15_28_22/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_31_41/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_41_23/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_34_50/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_30_37/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_26_43/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_38_12/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_07 14_51_07/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_07 14_30_16/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_08 14_39_35/test_sum_up.txt
./data/version_1/kenh14/log_dir/recommend/new/2023_07_07 15_01_44/test_sum_up.txt
./data/version_1

In [6]:
# Tách chuỗi thành các phần tử
def process_test_sum_up(content):
    result_list = content.split()

    # Lọc các phần tử chứa giá trị bạn cần
    result_dict = {}
    for i in range(len(result_list)-1):
        if result_list[i] in ['AUC', 'MRR', 'NDCG@3', 'NDCG@5', 'NDCG@10']:
            result_dict[result_list[i]] = result_list[i+1]

    # In từ điển kết quả
    return result_dict

In [7]:
valid_path[1].split('/')[-2]

'2023_07_09 03_40_07'

In [8]:
all_results = []
for i in range(0, len(valid_path) - 1, 2):
    with open(valid_path[i], 'r') as f:
        result = process_test_sum_up(f.read())
        result['date'] = valid_path[i].split('/')[-2]
        
    with open(valid_path[i+1], 'r') as f:
        content = f.read()
    
    configs = json.loads(content)
    configs.update(result)
    all_results.append(configs)

In [9]:
len(valid_path)

192

In [10]:
df = pd.DataFrame(all_results, columns=list(configs.keys()))

In [11]:
df = df[df['date'].str.contains('2023', regex=False)]

In [12]:
df = df[df['date'] > '2023_06_00 0_0_0']

In [13]:
df.columns

Index(['data_dir', 'hidden_dim', 'post_encoder_method',
       'additive_attention_query_vector_dim', 'text_encoder_method',
       'raw_text_encoder_method', 'raw_text_encoder_cnn_kernel_size',
       'raw_text_encoder_fastformer_num_heads',
       'raw_text_encoder_multi_head_self_attention_num_heads',
       'raw_text_encoder_transformer_num_heads',
       'pre_encoded_text_encoder_method', 'text_pre_encoded_dim',
       'category_encoder_method', 'category_embedding_dim', 'num_categories',
       'user_encoder_method', 'history_user_encoder_method',
       'history_user_encoder_fastformer_num_heads',
       'history_user_encoder_multi_head_self_attention_num_heads',
       'history_user_encoder_transformer_num_heads', 'history_length',
       'num_negative_samples', 'use_contrastive_learning', 'crop_rate',
       'insert_rate', 'mask_rate', 'reorder_rate', 'substitute_rate',
       'history_length_threshold', 'user_history_augmentation_method',
       'batch_size', 'recommend_learn

In [14]:
df[['date', 'data_dir', 'post_encoder_method', 'text_encoder_method', 'raw_text_encoder_method', \
    'user_encoder_method', 'history_user_encoder_method', 'use_contrastive_learning', 'user_history_augmentation_method', 'recommend_weight',\
    'AUC', 'MRR', 'NDCG@5', 'NDCG@10']].to_csv('./csv_result.csv')

In [15]:
df_test = df[['date', 'data_dir', 'post_encoder_method', 'text_encoder_method', 'raw_text_encoder_method', \
    'user_encoder_method', 'history_user_encoder_method', 'use_contrastive_learning', 'user_history_augmentation_method', 'recommend_weight',\
    'AUC', 'MRR', 'NDCG@5', 'NDCG@10']]

In [16]:
df_test = df_test.drop_duplicates(subset=['data_dir', 'post_encoder_method', 'text_encoder_method', 'raw_text_encoder_method', \
    'user_encoder_method', 'history_user_encoder_method', 'use_contrastive_learning', 'user_history_augmentation_method', 'recommend_weight',\
    'AUC', 'MRR', 'NDCG@5', 'NDCG@10'])

In [17]:
def smooth_float(number):
    return round(number, 1)

In [18]:
df_test['recommend_weight'] = df['recommend_weight'].apply(smooth_float)

In [25]:
df_test.sort_values(['data_dir', 'AUC'], ascending=[True, False]).to_csv('./csv_result_1.csv')

In [26]:
df_test[df_test['use_contrastive_learning'] == False].sort_values(['data_dir', 'raw_text_encoder_method'])

,date,data_dir,post_encoder_method,text_encoder_method,raw_text_encoder_method,user_encoder_method,history_user_encoder_method,use_contrastive_learning,user_history_augmentation_method,recommend_weight,AUC,MRR,NDCG@5,NDCG@10


In [27]:
df_test.sort_values('date')

,date,data_dir,post_encoder_method,text_encoder_method,raw_text_encoder_method,user_encoder_method,history_user_encoder_method,use_contrastive_learning,user_history_augmentation_method,recommend_weight,AUC,MRR,NDCG@5,NDCG@10
40,2023_07_06 15_30_14,data/version_1/kenh14,TitleAbtractCategorySubcategory,Raw,CNN,History,AdditiveAttention,True,Version1,1.0,0.7966,0.3408,0.4108,0.4957
58,2023_07_06 16_13_38,data/version_1/kenh14,TitleAbtractCategorySubcategory,Raw,CNN,History,Fastformer,True,Version1,1.0,0.8059,0.3609,0.4320,0.5140
39,2023_07_06 17_00_34,data/version_1/kenh14,TitleAbtractCategorySubcategory,Raw,CNN,History,MultiHeadSelfAttention,True,Version1,1.0,0.7924,0.3318,0.3930,0.4848
30,2023_07_06 17_46_48,data/version_1/kenh14,TitleAbtractCategorySubcategory,Raw,CNN,History,Transformer,True,Version1,1.0,0.8174,0.3578,0.4361,0.5206
38,2023_07_06 18_33_47,data/version_1/kenh14,TitleAbtractCategorySubcategory,Raw,Fastformer,History,AdditiveAttention,True,Version1,1.0,0.8174,0.3776,0.4577,0.5381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,2023_07_19 03_33_53,data/version_1/kenh14,TitleAbtractCategorySubcategory,PreEncoded,MultiHeadSelfAttention,History,Fastformer,True,Version2,0.4,0.8199,0.3812,0.4619,0.5410
7,2023_07_19 04_14_58,data/version_1/kenh14,TitleAbtractCategorySubcategory,PreEncoded,MultiHeadSelfAttention,History,Fastformer,True,Version2,0.5,0.8118,0.3734,0.4497,0.5259
37,2023_07_19 04_56_54,data/version_1/kenh14,TitleAbtractCategorySubcategory,PreEncoded,MultiHeadSelfAttention,History,Fastformer,True,Version2,0.6,0.8033,0.3646,0.4353,0.5169
60,2023_07_19 05_40_49,data/version_1/kenh14,TitleAbtractCategorySubcategory,PreEncoded,MultiHeadSelfAttention,History,Fastformer,True,Version2,0.7,0.8258,0.3918,0.4751,0.5518
